In [13]:
#!import config/Settings.cs

In [14]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.0.0-rc3"

Installed Packages Microsoft.SemanticKernel, 1.0.0-rc3

# 2.1 🚀 Initialization and Configuration

* In this section, libraries are imported, and OpenAI services are configured. The KernelBuilder is initialized to set up the Semantic Kernel. OpenAI models, including GPT-3.5 Turbo and GPT-4 32K, are loaded with their associated API keys and endpoints. Finally, the Semantic Kernel is built using the configured settings.

In [15]:
using Microsoft.SemanticKernel;

var builder = new KernelBuilder();

In [16]:
// Loading models and setting up OpenAI chat completion services
var (model1, azureEndpoint1, apiKey1, gpt35TurboServiceId) = Settings.LoadFromFile(model: "gpt-35-turbo");
builder.AddAzureOpenAIChatCompletion(model1, model1, azureEndpoint1, apiKey1, serviceId: gpt35TurboServiceId);

var (model2, azureEndpoint2, apiKey2, gpt4ServiceId) = Settings.LoadFromFile(model: "gpt-4-32k");
builder.AddAzureOpenAIChatCompletion(model2, model2, azureEndpoint2, apiKey2, serviceId: gpt4ServiceId);

var kernel = builder.Build();

# 2.2. 🚀 Semantic Kernel Pipeline: Calling Chain of Functions

* The pipeline begins by defining prompts for functions, including requests to find top books in world history and to translate text. Functions are then created based on these prompts. The pipeline is organized by specifying the order in which functions will be executed.


In [17]:
// Prompts initialization
string findBooksPrompt = @"I would sincerely appreciate your assistance in curating a list of the top {{$BooksNumber}} books that explore the fascinating realm of world history. 
It would be especially helpful if you could suggest books from the period {{$YearFrom}} to {{$YearTo}}. Show just Name and Author.";

string translatePrompt = "Your linguistic expertise is highly valued. Please translate the following text into {{$Lang}}. TEXT: {{$Input}}";

In [18]:
// Create function for finding top books in world history
KernelFunction findBooksFunction = kernel.CreateFunctionFromPrompt(
    findBooksPrompt,
    functionName: "TopBooks",
    description: "Retrieves a curated list of top books on world history within a specified time period.");

// Create function for translating text
KernelFunction translateFunction = kernel.CreateFunctionFromPrompt(
    translatePrompt,
    functionName: "Translate",
    description: "Translates the provided text into the specified language.");


In [19]:
// Set up a pipeline of functions to be executed
var pipeline = new KernelFunction[]
{
    findBooksFunction,
    translateFunction
};

# 2.3. 🚀 Function Arguments

* This module focuses on setting up specific arguments required for function execution. Parameters such as the number of books, date range, and target language for translation are defined to tailor the behavior of the functions.


In [20]:
using Microsoft.SemanticKernel.Connectors.AI.OpenAI;

// Set execution settings for OpenAI service
KernelArguments arguments = new KernelArguments
{
    ExecutionSettings = new OpenAIPromptExecutionSettings
    {
        ServiceId = gpt35TurboServiceId,
        Temperature = 0
    }
};

// Set specific arguments for the findBooks function
arguments.Add("BooksNumber", "3");
arguments.Add("YearFrom", "1950");
arguments.Add("YearTo", "1960");

// Set specific aguments for the translate function
arguments.Add("Lang", "Italian");

# 2.4. 🚀 Event Handlers

* Event handlers are established to capture and respond to events during Semantic Kernel execution. These handlers cover prompt rendering, prompt rendered, function invoking, and function invoked events, providing insights into the execution flow.


In [21]:
using Microsoft.SemanticKernel.Events;

kernel.FunctionInvoking += Kernel_FunctionInvoking;
kernel.FunctionInvoked += Kernel_FunctionInvoked;
kernel.PromptRendering += Kernel_PromptRendering;
kernel.PromptRendered += Kernel_PromptRendered;

// Define event handlers
void Kernel_PromptRendering(object? sender, PromptRenderingEventArgs e)
{
    Console.WriteLine($"[PromptRendering]: \n\t // {e.Function.Description} \n\t {e.Function.Name} ({string.Join(":", e.Arguments)})");
}

void Kernel_PromptRendered(object? sender, PromptRenderedEventArgs e)
{
    Console.WriteLine($"[PromptRendered]: \n\t{e.RenderedPrompt}");
}

void Kernel_FunctionInvoking(object? sender, FunctionInvokingEventArgs e)
{
    Console.WriteLine($"[FunctionInvoking]: \n\t // {e.Function.Description} \n\t {e.Function.Name} ({string.Join(":", e.Arguments)})");
}

void Kernel_FunctionInvoked(object? sender, FunctionInvokedEventArgs e)
{
    Console.WriteLine($"[FunctionInvoked]: \n\t {e.Function.Name}");
}

# 2.4. 🚀 Function Pipeline Invocation and Output

* Functions in the pipeline are invoked, and the results are displayed. This section showcases the step-by-step execution of the Semantic Kernel, allowing for a better understanding of its behavior and outputs.


In [22]:
// Invoke each function in the pipeline and display the output
var outputs = new List<string>();

foreach (var item in pipeline)
{
    var pipelineResult = await kernel.InvokeAsync(item, arguments);
    arguments["Input"] = pipelineResult;
    var valueAsString = pipelineResult.GetValue<string>();
    outputs.Add($"[{pipelineResult.Function.Name}]: {valueAsString}");
}

[FunctionInvoking]: 
	 // Retrieves a curated list of top books on world history within a specified time period. 
	 TopBooks ([BooksNumber, 3]:[YearFrom, 1950]:[YearTo, 1960]:[Lang, Italian])
[PromptRendering]: 
	 // Retrieves a curated list of top books on world history within a specified time period. 
	 TopBooks ([BooksNumber, 3]:[YearFrom, 1950]:[YearTo, 1960]:[Lang, Italian])
[PromptRendered]: 
	I would sincerely appreciate your assistance in curating a list of the top 3 books that explore the fascinating realm of world history. 
It would be especially helpful if you could suggest books from the period 1950 to 1960. Show just Name and Author.
[FunctionInvoked]: 
	 TopBooks
[FunctionInvoking]: 
	 // Translates the provided text into the specified language. 
	 Translate ([BooksNumber, 3]:[YearFrom, 1950]:[YearTo, 1960]:[Lang, Italian]:[Input, Finding specific books on world history published between 1950 and 1960 can be a bit challenging due to the broad nature of the topic and the s

In [23]:
Console.WriteLine(string.Join("\n", outputs));

[TopBooks]: Finding specific books on world history published between 1950 and 1960 can be a bit challenging due to the broad nature of the topic and the specific time frame. However, here are three influential books from around that period:

1. "The Rise and Fall of the Third Reich" by William L. Shirer (1960)
2. "The Guns of August" by Barbara W. Tuchman (1962)
3. "A Study of History" by Arnold J. Toynbee (1954)

Please note that the publication dates for some of these books are slightly outside of your specified range, but they are still highly regarded works from roughly the same era.
[Translate]: Trovare libri specifici sulla storia mondiale pubblicati tra il 1950 e il 1960 può essere un po' impegnativo a causa della vastità dell'argomento e del periodo di tempo specifico. Tuttavia, ecco tre libri influenti di quel periodo:

1. "L'ascesa e la caduta del Terzo Reich" di William L. Shirer (1960)
2. "Le pistole di agosto" di Barbara W. Tuchman (1962)
3. "Uno studio della storia" di A